# **Chapter 7: Using Examples (Few-Shot Prompting)**

---
**Lesson:**

Giving Llama2 examples of how you want it to behave (or how you want it not to behave) is extremely effective for:
- Getting the right answer
- Getting the answer in the right format

This sort of prompting is also called "few shot prompting".

---

First we will setup our dependencies

In [ ]:
%%capture
#Install dependencies
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install --quiet langchain==0.0.304

#Import libraries, and set up Bedrock client
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

modelId = 'meta.llama2-13b-chat-v1' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"

def invoke_model_and_get_response(prompt_data): 
    body = json.dumps({ 
        'prompt': prompt_data,
        'max_gen_len': 512,
        'top_p': 0.9,
        'temperature': 0.2
    })

    try:
        response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
        response_body = json.loads(response.get('body').read().decode('utf-8'))
        outputText = response_body['generation'].strip()
        return outputText

    except botocore.exceptions.ClientError as error:
        if error.response['Error']['Code'] == 'AccessDeniedException':
            return (f"\x1b[41m{error.response['Error']['Message']}\
                    \nTo troubleshoot this issue please refer to the following resources.\
                     \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                     \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        else:
            raise

# **Examples:**

**Example 7.1 - Is Santa real?**

Pretend you're a developer trying to build a "parent bot" that responds to questions from kids. Llama2's default response is quite formal and robotic. This is going to break a child's heart.

In [ ]:
# create the prompt
prompt_data = """
<s>[INST] 

<<SYS>>
You are an AI assistant that gives only the truth and reality.
<</SYS>>

Will Santa bring me presents on Christmas?[/INST]
"""

response = invoke_model_and_get_response(prompt_data)
print(response)

**Example 7.2 - Santa is real!**

To improve Llama2's response, let's allow Llama2 to think things out first before answering. We do that by literally spelling out the steps that Llama2 should take in order to process and think through the prompt. 
Now, Llama2's response is much better and more nuanced.

In [ ]:
# create the prompt
prompt_data = """
<s>[INST] 

<<SYS>>
<</SYS>>

Please complete the conversation by writing the next line, speaking as ""A"".
Q: Is the tooth fairy real?
A: Of course, sweetie. Wrap up your tooth and put it under your pillow tonight. There might be something waiting for you in the morning.
Q: Will Santa bring me preents on Christmas?[/INST]"""

response = invoke_model_and_get_response(prompt_data)
print(response)

**Example 7.3 - Extraction**

In the following formatting example, we could walk Llama2 step by step through a set of formatting instructions on how to extract names and professions and then format them exactly the way we want, or we could just provide Llama2 with some correctly-formatted examples and Llama2 can extrapolate from there.

In [ ]:
# create the prompt
prompt_data = """
<s>[INST] 

<<SYS>>
<</SYS>>

In the bustling town of Emerald Hills, a diverse group of individuals made their mark. Sarah Martinez, a dedicated nurse, was known for her compassionate care at the local hospital. David Thompson, an innovative software engineer, worked tirelessly on groundbreaking projects that would revolutionize the tech industry. Meanwhile, Emily Nakamura, a talented artist and muralist, painted vibrant and thought-provoking pieces that adorned the walls of buildings and galleries alike. Lastly, Michael O'Connell, an ambitious entrepreneur, opened a unique, eco-friendly cafe that quickly became the town's favorite meeting spot. Each of these individuals contributed to the rich tapestry of the Emerald Hills community.
<individuals>
1. Sarah Martinez [NURSE]
2. David Thompson [SOFTWARE ENGINEER]
3. Emily Nakamura [ARTIST]
4. Michael O'Connell [ENTREPRENEUR]
</individuals>

At the heart of the town, Chef Oliver Hamilton has transformed the culinary scene with his farm-to-table restaurant, Green Plate. Oliver's dedication to sourcing local, organic ingredients has earned the establishment rave reviews from food critics and locals alike.

Just down the street, you'll find the Riverside Grove Library, where head librarian Elizabeth Chen has worked diligently to create a welcoming and inclusive space for all. Her efforts to expand the library's offerings and establish reading programs for children have had a significant impact on the town's literacy rates.

As you stroll through the charming town square, you'll be captivated by the beautiful murals adorning the walls. These masterpieces are the work of renowned artist, Isabella Torres, whose talent for capturing the essence of Riverside Grove has brought the town to life.

Riverside Grove's athletic achievements are also worth noting, thanks to former Olympic swimmer-turned-coach, Marcus Jenkins. Marcus has used his experience and passion to train the town's youth, leading the Riverside Grove Swim Team to several regional championships.
<individuals>
1. Oliver Hamilton [CHEF]
2. Elizabeth Chen [LIBRARIAN]
3. Isabella Torres [ARTIST]
4. Marcus Jenkins [COACH]
</individuals>

Oak Valley, a charming small town, is home to a remarkable trio of individuals whose skills and dedication have left a lasting impact on the community.

At the town's bustling farmer's market, you'll find Laura Simmons, a passionate organic farmer known for her delicious and sustainably grown produce. Her dedication to promoting healthy eating has inspired the town to embrace a more eco-conscious lifestyle.

In Oak Valley's community center, Kevin Alvarez, a skilled dance instructor, has brought the joy of movement to people of all ages. His inclusive dance classes have fostered a sense of unity and self-expression among residents, enriching the local arts scene.

Lastly, Rachel O'Connor, a tireless volunteer, dedicates her time to various charitable initiatives. Her commitment to improving the lives of others has been instrumental in creating a strong sense of community within Oak Valley.

Through their unique talents and unwavering dedication, Laura, Kevin, and Rachel have woven themselves into the fabric of Oak Valley, helping to create a vibrant and thriving small town.

Please return a numbered list like the other 2 examples provided.[/INST]"""

response = invoke_model_and_get_response(prompt_data)
print(response)

**Example 7.4 - Copycat**

For simple tasks, you may not even need any instructions; examples can be enough. Of course, best practice would be to include instructions anyway. However, this example below is meant to show off the power of examples.

In [ ]:
# create the prompt
prompt_data = """
<s>[INST] 

<<SYS>>
<</SYS>>

Quote:
“When the reasoning mind is forced to confront the impossible again and again, it has no choice but to adapt.”
― N.K. Jemisin, The Fifth Season
Author: N.K. Jemisin

Quote:
“Some humans theorize that intelligent species go extinct before they can expand into outer space. If they're correct, then the hush of the night sky is the silence of the graveyard.”
― Ted Chiang, Exhalation
Author:
[/INST]"""

response = invoke_model_and_get_response(prompt_data)
print(response)

# Chapter 7 - END.